In [1]:
import os

In [2]:
# Current folder path
%pwd

'/Volumes/Macintosh/mlflow-dvc-food-spoilage-detector/notebooks'

In [3]:
# Go to the roor directory and give the path
os.chdir("../")
%pwd

'/Volumes/Macintosh/mlflow-dvc-food-spoilage-detector'

In [4]:
from dataclasses import dataclass
from pathlib import Path
from constants import *
from utils.base_utils import read_yaml, create_directories

In [5]:
# == Entity ==
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
# Configuration Manager ==
class ConfigurationManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training_config = self.config.training
        prepare_base_model_config = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(
            self.config.data_preprocessing.root_dir, "Food_Spoilage_Dataset"
        )

        create_directories([training_config.root_dir])

        training_config = TrainingConfig(
            root_dir=Path(training_config.root_dir),
            trained_model_path=Path(training_config.trained_model_path),
            updated_base_model_path=Path(
                prepare_base_model_config.updated_base_model_path
            ),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
        )

        print(training_config)
        return training_config

In [7]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
except Exception as e:
    raise e

2025-12-18 21:52:12,409 | fsd_logger | INFO | base_utils.py:35 | YAML file config/config.yaml loaded successfully
2025-12-18 21:52:12,411 | fsd_logger | INFO | base_utils.py:35 | YAML file params.yaml loaded successfully
2025-12-18 21:52:12,412 | fsd_logger | INFO | base_utils.py:59 | Directory created at artifacts
2025-12-18 21:52:12,412 | fsd_logger | INFO | base_utils.py:59 | Directory created at artifacts/training
TrainingConfig(root_dir=PosixPath('artifacts/training'), trained_model_path=PosixPath('artifacts/training/model.h5'), updated_base_model_path=PosixPath('artifacts/prepare_base_model/updated_base_model.h5'), training_data=PosixPath('artifacts/data_preprocessing/Food_Spoilage_Dataset'), params_epochs=1, params_batch_size=16, params_is_augmentation=True, params_image_size=BoxList([224, 244, 3]))
